In [3]:
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import explained_variance_score
from scipy import stats
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearnex import patch_sklearn
from sklearn.model_selection import cross_val_score
from sklearn.datasets import load_iris
import numpy as np

patch_sklearn()

X_train_fr = pd.read_csv('x_train_fr.csv')
X_train_de = pd.read_csv('x_train_de.csv')
X_test_fr = pd.read_csv('x_test_fr.csv')
X_test_de = pd.read_csv('x_test_de.csv')
y_test_fr = pd.read_csv('y_test_fr.csv')
y_test_de = pd.read_csv('y_test_de.csv')
y_train_fr = pd.read_csv('y_train_fr.csv')
y_train_de = pd.read_csv('y_train_de.csv')

X_test_de.head()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


,ID,DAY_ID,COUNTRY,DE_CONSUMPTION,FR_CONSUMPTION,FR_DE_EXCHANGE,DE_NET_IMPORT,FR_NET_IMPORT,DE_GAS,FR_GAS,...,FR_RESIDUAL_LOAD,DE_RAIN,FR_RAIN,DE_WIND,FR_WIND,DE_TEMP,FR_TEMP,GAS_RET,COAL_RET,CARBON_RET
0,730,1068,DE,1.458140,0.655233,-0.410717,-1.284736,-0.450144,-0.208664,0.434705,...,-0.169851,-0.204061,-0.219260,7.149098,3.216234,3.113181,2.139785,-0.211592,-1.165030,0.914272
1,273,959,DE,-0.014648,-0.873181,2.081281,1.108016,-1.208788,-0.410501,-1.230979,...,-1.504244,2.156969,5.218516,0.117698,0.445306,-1.601896,-1.370031,0.408783,-0.369350,-0.738291
2,911,499,DE,-0.800945,-1.390917,-0.302098,-0.793661,-0.411873,-0.734524,-0.043101,...,-2.096396,-0.192578,0.683547,1.384888,2.358903,1.433337,0.906934,1.053799,-1.363392,-0.726740
3,274,1050,DE,-0.132038,-0.697277,1.299494,1.628553,-1.093077,0.735354,-0.390132,...,-0.809076,-0.565251,-0.300878,-0.789039,-0.538239,-2.434426,-1.554886,-1.214039,-0.734142,-2.341894
4,619,291,DE,-0.056473,-0.728421,0.340486,-0.057127,-0.916117,0.175566,0.068640,...,-0.688561,-0.533701,-0.817087,0.006579,0.787396,-0.163353,0.422565,0.453732,0.103504,1.572016


In [4]:
X_train_fr.drop(columns=['COUNTRY'], inplace=True)
X_test_fr.drop(columns=['COUNTRY'], inplace=True)
y_true_fr = y_test_fr.iloc[:, 1]
y_train_fr_1 = y_train_fr.iloc[:, 1]

In [5]:
X_train_de.drop(columns=['COUNTRY'], inplace=True)
X_test_de.drop(columns=['COUNTRY'], inplace=True)
y_true_de = y_test_de.iloc[:, 1]
y_train_de_1 = y_train_de.iloc[:, 1]

In [ ]:
GBR = GradientBoostingRegressor()

param_grid = {
    'n_estimators': [10, 20, 30, 50, 100, 200],
    'learning_rate': [0.0001, 0.001, 0.005, 0.01, 0.1, 0.2],
    'max_depth': [3, 4, 5]
}

grid_search = GridSearchCV(estimator=GBR, param_grid=param_grid, cv=3, n_jobs=-1, scoring='neg_mean_squared_error')

grid_search.fit(X_train_fr, y_train_fr)
GBR = grid_search.best_estimator_

print("\nBest Parameters:", grid_search.best_params_)

In [6]:
GBR_fr = GradientBoostingRegressor(learning_rate= 0.001, max_depth= 3, n_estimators= 40)

GBR_fr.fit(X_train_fr, y_train_fr_1)
y_pred_gradient_boosting = GBR_fr.predict(X_test_fr)


MAE_gradient_boosting = mean_absolute_error(y_true_fr, y_pred_gradient_boosting)
MSE_gradient_boosting = mean_squared_error(y_true_fr, y_pred_gradient_boosting)

Corr_gradient_boosting, _ = stats.spearmanr(y_true_fr, y_pred_gradient_boosting)

print("Evaluation Metrics test:")
print("MAE:", MAE_gradient_boosting)
print("MSE:", MSE_gradient_boosting)
print("Spearman Correlation:", Corr_gradient_boosting)

Evaluation Metrics test:
MAE: 0.8531950389428022
MSE: 1.1237552880557584
Spearman Correlation: 0.03143455711799676


In [7]:
y_train_gradient_boosting_fr = GBR_fr.predict(X_train_fr)


MAE_gradient_boosting = mean_absolute_error(y_train_fr_1, y_train_gradient_boosting_fr)
MSE_gradient_boosting = mean_squared_error(y_train_fr_1, y_train_gradient_boosting_fr)

Corr_gradient_boosting, _ = stats.spearmanr(y_train_fr_1, y_train_gradient_boosting_fr)

print("Evaluation Metrics Train:")
print("MAE:", MAE_gradient_boosting)
print("MSE:", MSE_gradient_boosting)
print("Spearman Correlation:", Corr_gradient_boosting)

Evaluation Metrics Train:
MAE: 0.5001871023627414
MSE: 1.0376406768291284
Spearman Correlation: 0.13007542979237


In [ ]:
GBR_de = GradientBoostingRegressor()

param_grid = {
    'n_estimators': [10, 20, 30, 50, 100, 200],
    'learning_rate': [0.0001, 0.001, 0.005, 0.01, 0.1, 0.2],
    'max_depth': [3, 4, 5]
}

grid_search = GridSearchCV(estimator=GBR_de, param_grid=param_grid, cv=3, n_jobs=-1, scoring='neg_mean_squared_error')

grid_search.fit(X_train_de, y_train_de_1)
GBR_de = grid_search.best_estimator_

print("\nBest Parameters:", grid_search.best_params_)

In [8]:
GBR_de_best = GradientBoostingRegressor(learning_rate= 0.008, max_depth= 8, n_estimators= 30)

GBR_de_best.fit(X_train_de, y_train_de_1)
y_pred_gradient_boosting_de = GBR_de_best.predict(X_test_de)


MAE_gradient_boosting = mean_absolute_error(y_true_de, y_pred_gradient_boosting_de)
MSE_gradient_boosting = mean_squared_error(y_true_de, y_pred_gradient_boosting_de)

Corr_gradient_boosting, _ = stats.spearmanr(y_true_de, y_pred_gradient_boosting_de)

print("Evaluation Metrics Test:")
print("MAE:", MAE_gradient_boosting)
print("MSE:", MSE_gradient_boosting)
print("Spearman Correlation:", Corr_gradient_boosting)

Evaluation Metrics Test:
MAE: 0.8110972538672294
MSE: 0.9986218781359781
Spearman Correlation: 0.23752545615152984


In [10]:
y_train_gradient_boosting_de = GBR_de_best.predict(X_train_de)


MAE_gradient_boosting = mean_absolute_error(y_train_de_1, y_train_gradient_boosting_de)
MSE_gradient_boosting = mean_squared_error(y_train_de_1, y_train_gradient_boosting_de)

Corr_gradient_boosting, _ = stats.spearmanr(y_train_de_1, y_train_gradient_boosting_de)

print("Evaluation Metrics Train:")
print("MAE:", MAE_gradient_boosting)
print("MSE:", MSE_gradient_boosting)
print("Spearman Correlation:", Corr_gradient_boosting)

Evaluation Metrics Train:
MAE: 0.5878217926488171
MSE: 0.8477292595912487
Spearman Correlation: 0.634531486645206


In [18]:
import numpy as np
Y_val_combined = np.concatenate((y_true_fr, y_true_de))
Y_pred_combined = np.concatenate((y_pred_gradient_boosting, y_pred_gradient_boosting_de))

mse_combined = mean_squared_error(Y_val_combined, Y_pred_combined)
Corr_gradient_boosting, _ = stats.spearmanr(Y_val_combined, Y_pred_combined)

print(f"Combined MSE with best parameters: {mse_combined}")
print("Spearman Correlation:", Corr_gradient_boosting)

Combined MSE with best parameters: 1.0684593316844793
Spearman Correlation: 0.08616423730453814


In [19]:
Y_val_combined_test = np.concatenate((y_train_fr_1, y_train_de_1))
Y_pred_combined_test = np.concatenate((y_train_gradient_boosting_fr, y_train_gradient_boosting_de))

mse_combined = mean_squared_error(Y_val_combined_test, Y_pred_combined_test)
Corr_gradient_boosting, _ = stats.spearmanr(Y_val_combined_test, Y_pred_combined_test)

print(f"Combined MSE with best parameters: {mse_combined}")
print("Spearman Correlation:", Corr_gradient_boosting)

Combined MSE with best parameters: 0.9559050400928789
Spearman Correlation: 0.3368753083136654
